# Lab 4: Build a RAG Application with LangChain, Part 3

In this lab we continue to build on [Lab2](./2_rag.ipynb) and [Lab 3](./3_rag-with-chunking.ipynb)

Learning Objectives

* Learn how to use Azure AI Search for a Vector Store
* Learn how to add citations to the response

### Step 1: Setup what we covered in Lab 2 and Lab 3

Run the following to get ready for this lesson:

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd

load_dotenv()

llm = AzureChatOpenAI(
  openai_api_version="2023-05-15",
  azure_deployment= os.getenv("AZURE_OPENAI_MODEL_DEPLOYMENT_NAME")
)

embeddings = AzureOpenAIEmbeddings()

parser = StrOutputParser()

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a helpful assistant that is very brief but polite in your answers. Answer questions in less than 50 words.
            Answer the question based on the context below. If you can't 
            answer the question, reply "I don't know".

            Context: {context}
         """),
        ("human", "{question}")
    ],
)

DATASET_NAME = "./prep/output/master.json"
transcripts_dataset = pd.read_json(DATASET_NAME)

loader = DataFrameLoader(transcripts_dataset, page_content_column="text")
transcripts = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(transcripts)

See how many documents we are working with in this lab:

In [ ]:
len(documents)

### Step 2: Use Azure AI Search as the vector store

**Please change the index name to include your initials or name so it won't have duplicate entries from another user in it**

Run the below to configure the client used to interact with Azure Search

In [ ]:
from langchain.vectorstores.azuresearch import AzureSearch

vectorstore_address = os.getenv("AZURE_SEARCH_ENDPOINT")
vectorstore_password = os.getenv("AZURE_SEARCH_KEY")

# Add your name to the index name to avoid conflicts with other users
index_name = "bos-gab-index-<yourname>"
vectorstore = AzureSearch(
    azure_search_endpoint=vectorstore_address,
    azure_search_key=vectorstore_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

Now let's add the documents loaded in Step 1 into the AzureSearch vector store.

> NOTE: This will take a few minutes depending on the network speed.

In [ ]:
vectorstore.add_documents(documents=documents)

Now there are documents to use, try a similarity search to verify things are working:

In [ ]:
docs = vectorstore.similarity_search(
    query="What is langchain?",
    k=3,
    search_type="similarity",
)
docs

Ok, it seems to be working. Now let's move toward the functionality we testing in Lab 3.

Create a retriever to use later:

In [ ]:
retriever = vectorstore.as_retriever()

Test to see if it is working:

In [ ]:
unique_docs = retriever.get_relevant_documents(query="What is langchain?")
unique_docs

Now just like at the end of Lab 3, try out it out - just remember the vector store is not running in-memory this time, but in Azure.

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | parser
)

chain.invoke("What is LangChain?")

### Step 3: Add citations for identifying the source

One of the important parts of a RAG application is to include citations - this helps verify the LLM is not making something up.

> NOTE: Citations do not guarantee the LLM didn't make up the response, but it does help to verify to help you with a confidence level.

In order to have the response include citations we need to do a couple of things:
1. Modify the document listing used as the context to include the title of the video
1. Modify the prompt with some instructions for how to use the title and how we want it returned

The first thing we need is a utility function we'll use to modify the string returned from the retriever so the format will look like: 
```code
"<video title>:<transcript text>"
```

Run the following to declare the utility function:

In [ ]:
def format_docs(docs):
    return "\n\n".join([f"{d.metadata['title']}:{d.page_content}" for d in docs])

Next, we need to modify the prompt template to let the LLM know how the context is formatted and how to use the title and content:

In [ ]:
prompt_template_with_citations = ChatPromptTemplate.from_messages(
    [
        ("system", """Assistant helps people with their questions about the content of video transcripts. Be brief in your answers.
        Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. 
        Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
        Each source has a title followed by colon and the actual information, always include the source title for each fact you use in the response. 
         Use square brackets to reference the source, for example [Video title here]. Don't combine sources, list each source separately, for example [Video 1][Video 2].
            Context: {context}
         """),
        ("human", "{question}")
    ],
)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template_with_citations
    | llm
    | parser
)

Now give it a try:

In [ ]:
chain.invoke("What is langchain?")

In [ ]:
chain.invoke("What is AKS?")

### References

Here are some good resources to learn more about using Azure AI Search with LangChain:
* [Azure Cognitive Search and LangChain: A Seamless Integration for Enhanced Vector Search Capabilities](https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/azure-cognitive-search-and-langchain-a-seamless-integration-for/ba-p/3901448)
* LangChain Docs [Azure AI Search](https://python.langchain.com/docs/integrations/vectorstores/azuresearch/)
* [Azure AI Search client library for Python - version 11.4.0](https://learn.microsoft.com/en-us/python/api/overview/azure/search-documents-readme?view=azure-python)

## [Go To Next Lab](./5_rag-final.ipynb.ipynb)